In [ ]:
# Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

In [398]:
# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 1000)")
    time.sleep(1)

In [ ]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")  # 윈도우는 "chromedriver.exe"
url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
&offset={}".format(page)
    
driver.get(url)
time.sleep(2)

# 데이터 크롤링

In [456]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")  # 윈도우는 "chromedriver.exe"
url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
&offset=0"
driver.get(url)
time.sleep(2)

hotel_list = []
url_list=[]
rating_list = []
price_list = []
room_list = []

print("크롤링 성공: ")
print('hotel', 'rating', 'price', 'room', 'url')

page = 10   # 크롤링 원하는 페이지 수
for i in range(0, page):
    
    # 호텔 이름
    hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
    for hotel in hotel_raw:
        hotel = hotel.text
        hotel_list.append(hotel)
    # hotel_list = [ x for x in hotel_list if not x =='']
    
    # 별점 가져오기
    rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
    for rating in rating_raw:
        if '\n' in rating.text:
            rating = rating.text.split('\n')[0]
        else:
            rating = rating.text
        rating_list.append(rating)
    
    # 가격 가져오기
    price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
    for price in price_raw:
        if price.text.split('\n') != ['']:
            price = price.text.split('\n')
            price = [ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1]
            price_list.append(price)
    price_list = [x for x in price_list if not x =='']
    
    # 방 정보
    room_raw = driver.find_elements_by_css_selector(".room_link")
    for room in room_raw:
        room_list.append(room.text.split('\n')[0].replace(' -','').strip())
    room_list = [ x for x in room_list if not x =='']
    
    # url
    url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")
    for url in url_raw:
        url = url.get_attribute('href')
        url_list.append(url)
    time.sleep(1)
    # url_list = [ x for x in url_list if not x =='']
    
    # 크롤링 성공한 갯수
    print(len(hotel_list),len(rating_list),len(price_list),len(room_list),len(url_list))
    
    # 다음 페이지 이동
    driver.find_element_by_css_selector('.bk-icon.-iconset-navarrow_right.bui-pagination__icon').click()
    time.sleep(3)
    scroll_down(driver)
    time.sleep(2)
    
print("크롤링 완료")

크롤링 성공: 
hotel_list rating_list price_list room_list url_list
25 25 25 25 25
hotel_list rating_list price_list room_list url_list
50 50 50 50 50
hotel_list rating_list price_list room_list url_list
75 75 75 75 75
hotel_list rating_list price_list room_list url_list
100 100 100 100 100
hotel_list rating_list price_list room_list url_list
125 125 125 125 125
hotel_list rating_list price_list room_list url_list
150 150 150 150 150
hotel_list rating_list price_list room_list url_list
175 175 175 175 175
hotel_list rating_list price_list room_list url_list
200 200 200 200 200
hotel_list rating_list price_list room_list url_list
225 225 225 225 225
hotel_list rating_list price_list room_list url_list
250 250 250 250 250
크롤링 완료


In [457]:
print(len(hotel_list))
print(len(room_list))
print(len(price_list))
print(len(rating_list))
print(len(url_list))

250
250
250
250
250


In [458]:
# 데이터 프레임으로 만들기
df = pd.DataFrame({'hotel':hotel_list, 'rating':rating_list, 'room':room_list, 'url':url_list, 'price':price_list})
df    

,hotel,rating,room,url,price
0,신신호텔 서귀포,8.6,Standard Ondol Room(Single use),https://www.booking.com/hotel/kr/sinsinhotel-s...,"₩39,980"
1,롯데호텔 제주,8.6,한실 패밀리룸,https://www.booking.com/hotel/kr/lotte-jeju.ko...,"₩514,552"
2,랜딩관 제주신화월드 호텔앤리조트,8.3,수페리어 트윈룸,https://www.booking.com/hotel/kr/jeju-shinhwa-...,"₩220,000"
3,메종 글래드 제주,8.1,디럭스 트윈룸,https://www.booking.com/hotel/kr/jeju-grand.ko...,"₩175,500"
4,서머셋 제주신화월드,8.6,패밀리 스위트룸 한실 (온돌),https://www.booking.com/hotel/kr/somerset-jeju...,"₩660,000"
...,...,...,...,...,...
245,보우힐 리조트 & 펜션,9.0,스튜디오 (성인 3명),https://www.booking.com/hotel/kr/windhill-reso...,"₩62,790"
246,브릿지 레지던스 호텔,9.3,스탠다드 더블룸,https://www.booking.com/hotel/kr/beurisjirejid...,"₩45,000"
247,써니 데이 제주,9.3,디럭스 스튜디오,https://www.booking.com/hotel/kr/sunny-day-jej...,"₩144,000"
248,제주도 하숙집,9.4,남성 전용 도미토리룸 내 이층침대,https://www.booking.com/hotel/kr/jejudo-hasugj...,"₩20,020"


In [459]:
# 엑셀로 저장하기
df.to_excel("booking_com.xlsx", encoding='utf-8-sig')

# (참고 코드)

In [404]:
# 호텔 이름
hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
hotel_list = []
for hotel in hotel_raw:
    hotel = hotel.text
    hotel_list.append(hotel)
    
# hotel_list = [ x for x in hotel_list if not x =='']
print(len(hotel_list))
hotel_list

25


['호텔 더원',
 '빨간풍차펜션',
 '중문시티호텔',
 '베니키아 중문 호텔',
 'Floral Hotel Haema',
 '제주 해호텔',
 '탑스텐 빌라 드 애월',
 '그림리조트',
 '호텔 화이트 하우스',
 '엘린 호텔',
 '아이진 호텔',
 '에코 그린 리조트 & 풀 빌라',
 '오션 그랜드 호텔',
 '제주 디 오션 힐 펜션',
 '제주 올레요 리조트',
 'J2 패밀리 호텔',
 '제주 팔레스호텔',
 '수앤수 리조트',
 '제주 모슬포 호텔',
 '엘로망 펜션',
 '호텔 서귀피안',
 '호텔 앨리스 앤 트렁크',
 '하이 레지던스',
 '더 스테이 센츄리 호텔',
 '호텔 윈스토리']

In [336]:
# url 
url_list=[]
url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")

for url in url_raw:
    url = url.get_attribute('href')
    url_list.append(url)
    
# url_list = [ x for x in url_list if not x =='']
print(len(url_list))
url_list

25


['https://www.booking.com/hotel/kr/thira.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=33206dff5c436860f6918613600ee658&all_sr_blocks=263598007_271762592_0_0_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&group_adults=1&group_children=0&hapos=51&highlighted_blocks=263598007_271762592_0_0_0&hpos=1&no_rooms=1&req_adults=1&req_children=0&room1=A&sr_order=popularity&sr_pri_blocks=263598007_271762592_0_0_0__5576500&srepoch=1620641799&srpvid=9fe24843730e0017&ucfs=1&from=searchresults#hotelTmpl',
 'https://www.booking.com/hotel/kr/robero-jeju.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=33206dff5c436860f6918613600ee658&all_sr_blocks=135458058_303330955_2_0_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&

In [420]:
# 별점 가져오기
rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
rating_list = []
for rating in rating_raw:
    if '\n' in rating.text:
        rating = rating.text.split('\n')[0]
    else:
        rating = rating.text
    rating_list.append(rating)
    
# rating_list = [ x for x in rating_list if not x =='']
print(len(rating_list))
rating_list

25


['8.7',
 '7.1',
 '7.1',
 '9.7',
 '9.1',
 '7.7',
 '8.0',
 '6.8',
 '9.1',
 '',
 '5.8',
 '5.3',
 '',
 '8.8',
 '8.8',
 '7.7',
 '',
 '8.2',
 '8.9',
 '7.8',
 '7.8',
 '9.0',
 '9.3',
 '9.3',
 '9.4']

In [454]:
# 가격 가져오기
price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
price_list = []
for price in price_raw:
    if price.text.split('\n') != ['']:
        price = price.text.split('\n')
        price = [ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1]
        price_list.append(price)
price_list = [x for x in price_list if not x =='']
print(len(price_list))
price_list

25


['₩20,900',
 '₩51,300',
 '₩25,000',
 '₩25,000',
 '₩59,850',
 '₩50,000',
 '₩60,800',
 '₩25,000',
 '₩75,050',
 '₩31,350',
 '₩46,550',
 '₩38,000',
 '₩50,000',
 '₩45,000',
 '₩20,900',
 '₩50,696',
 '₩40,000',
 '₩346,332',
 '₩24,225',
 '₩23,750',
 '₩60,588',
 '₩40,000',
 '₩35,000',
 '₩25,000',
 '₩106,875']

In [376]:
# 방 정보
room_raw = driver.find_elements_by_css_selector(".room_link")
room_list = []
for room in room_raw:
    room_list.append(room.text.split('\n')[0].replace(' -','').strip())
    
room_list = [ x for x in room_list if not x =='']
print(len(room_list))
room_list

25


['스탠다드 더블룸',
 '스탠다드 트리플룸',
 '스탠다드 더블룸',
 '스탠다드 트윈룸',
 '객실 체크인 시 배정',
 '특별 혜택 트윈룸 산 전망, 2인 무료 조식',
 '스탠다드 트윈룸 산 전망',
 '스위트룸 전용 온수 수영장',
 '스탠다드 더블룸',
 '싱글룸',
 '더블룸',
 '패밀리 스위트룸   × 6',
 '스탠다드 트윈룸',
 '스탠다드 싱글룸',
 '트윈룸 바다 전망',
 '스탠다드 트윈룸',
 '트윈룸',
 '패밀리 트윈룸 바다 전망',
 '디럭스 트윈룸 산 전망',
 '스탠다드룸',
 'Standard Twin Room Ondol type',
 '수페리어 트윈룸',
 '스탠다드 싱글룸',
 '스탠다드 더블룸',
 '수페리어 트윈룸']